# Disciplina Design e Visualização II - MBA JDD
Jan/2025 - Tabalho Final

Estudante: Ícaro Ferracini

## Objetivo

O trabalho final da disciplina Design e Visualização II envolve criar uma página web simples com recursos interativos.

Desenvolver uma reportagem simples com quatro gráficos interativos
- Precisa ter título, linha fina, data de publicação, nome do autor
- Ter ao menos quatro parágrafos e uma imagem
- A imagem precisa estar em domínio público, ser autoral ou ter licença Creative Commons
- Obrigatoriamente, os gráficos interativos só poderão ser exibidos após uma ação do usuário como clique, toque, seleção ou input.
- Os gráficos poderão ser produzidos com Datawrapper ou Flourish — não recomenda-se misturar as ferramentas para evitar inconsistências visuais

Os estudantes devem:
- Escolher uma base de dados;
- Realizar o tratamento e análise de dados;
- Escolher um recorte (temporal, geográfico ou outros) para os quatro gráficos;
- Prototipação do layout e dos gráficos;
- Criar os gráficos em uma plataforma de dataviz no-code (Flourish ou Datawrapper);
- Escrever uma análise com base nos dados apurados;
- Escrever metodologia no pé da reportagem;
- Desenvolver uma página em HTML, CSS e JavaScript;
- Publicar o código da análise no GitHub ou Google Colab;
- Publicar a página no GitHub Pages;

Os trabalhos precisam ser mobile-first (desenvolvido primeiro para celular). Caso o estudante opte por deixar a página responsiva poderá ganhar até 2 pontos extras. Considero mobile a largura de 412px e 1024px para desktop.

Critérios avaliativos disponíveis em: https://whimsical.com/idp-2024-design-e-visualizacao-ii-8ZGJoamuwxpYvpcAS3CwpZ

# Análise da base de dados

O tema escolhido para o trabalho foi acessibilidade nos museus do Brasil.

Para isso foi obtida uma base de museus disponível em: https://cadastro.museus.gov.br/painel-analitico/

Importando bibliotecas

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs 
import urllib3
urllib3.disable_warnings()

Carregando a base

In [5]:
df = pd.read_csv('lista_museus.csv', sep=',', encoding='UTF-8')
df.head()

,ID,Título,Esfera,Status de Funcionamento,Tipo,Temática,Estado,Município,Site Oficial,Link no MuseusBR
0,80119,Casa da Cultura Dide Brandão,PÚBLICA – Municipal,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",SC,Itajaí,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_...
1,80120,Museu Sacro João XXIII,PRIVADA – Não informada,Aberto,Clássico/Tradicional,História,RN,Macaíba,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
2,80121,Museu Paranaense de Ciências Forenses,PÚBLICA – Estadual/Distrital,Aberto,Clássico/Tradicional,Defesa e segurança pública,PR,Curitiba,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_...
3,117440,Museu da Cultura Hip Hop RS,PRIVADA – Associação,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",RS,Porto Alegre,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
4,80122,"Museu Histórico, Antropológico, Arqueológico e...",PÚBLICA – Municipal,Aberto,Clássico/Tradicional,História,RS,Torres,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...


## Prompt para Chat GPT criar código para raspagem de dados:

No dataframe df existem as seguintes colunas:
- "ID" = um número de identificação como 80121
- "Link no MuseusBR" = uma URL como https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80121

Quero raspar os dados de cada URL da coluna Link no MuseusBR.
Eu preciso encontrar e salvar as seguintes informações em cada URL em um novo dataframe:

- div com a class "metadata-slug-acessibilidade_fisica" = Todo o texto da tag "p" em uma coluna da tabela
- div com a class "metadata-slug-acessibilidade_fisica" = Todo o texto das tags "a", separados por "|" em outra coluna da tabela
- div com a class "metadata-slug-outras-infraestruturas-para-pessoas-com-dificuldade-de-locomocao" = Todo o texto da tag "p" em uma outra coluna da tabela
- div com a class "metadata-slug-outras-infraestruturas-para-pessoas-com-dificuldade-de-locomocao" = Todo o texto das tags "a", separados por "|" em outra coluna da tabela
- div com a class "metadata-slug-infraestrutura-para-pessoas-com-deficiencias-auditivas-e-ou-visuais" = Todo o texto da tag "p" em uma outra coluna da tabela
- div com a class "metadata-slug-infraestrutura-para-pessoas-com-deficiencias-auditivas-e-ou-visuais" = Todo o texto das tags "a", separados por "|" em outra coluna da tabela
- div com a class "metadata-slug-outras-infraestrutura-para-pessoas-com-deficiencias-auditivas-e-ou-visuais" = Todo o texto da tag "p" em uma outra coluna da tabela
- div com a class "metadata-slug-outras-infraestrutura-para-pessoas-com-deficiencias-auditivas-e-ou-visuais" = Todo o texto das tags "a", separados por "|" em outra coluna da tabela

Salve o ID da URL na última coluna do dataframe.
Trate o código para, caso não exista a "div", "p" ou "a" correspontende na página, o valor seja retornado como "--"

---

OBS: O objetivo era pegar tanto parágrafos quanto links para verificar se há informação divergente. Podemos desconsiderar isso depois no tratamento de dados.

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Função para extrair texto das tags "p" e "a" dentro de uma div com classe específica
def extract_info(soup, class_name):
    div = soup.find("div", class_=class_name)
    if div:
        p_text = div.find("p").get_text(strip=True) if div.find("p") else "--"
        a_texts = "|".join([a.get_text(strip=True) for a in div.find_all("a")]) if div.find_all("a") else "--"
    else:
        p_text = "--"
        a_texts = "--"
    return p_text, a_texts

# Carregar o DataFrame com os links
df = pd.read_csv('lista_museus.csv', sep=',', encoding='UTF-8')

# Novo DataFrame para armazenar os resultados
columns = [
    "ID", 
    "Acessibilidade_Fisica_P", "Acessibilidade_Fisica_A", 
    "Outras_Infraestruturas_Locomocao_P", "Outras_Infraestruturas_Locomocao_A",
    "Infraestrutura_Def_Auditivas_Visuais_P", "Infraestrutura_Def_Auditivas_Visuais_A",
    "Outras_Infraestruturas_Auditivas_Visuais_P", "Outras_Infraestruturas_Auditivas_Visuais_A"
]
result_df = pd.DataFrame(columns=columns)

# Loop sobre as URLs
for index, row in df.iterrows():
    url = row['Link no MuseusBR']
    museum_id = row['ID']
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extração dos dados
        acessibilidade_fisica_p, acessibilidade_fisica_a = extract_info(soup, "metadata-slug-acessibilidade_fisica")
        outras_locomocao_p, outras_locomocao_a = extract_info(soup, "metadata-slug-outras-infraestruturas-para-pessoas-com-dificuldade-de-locomocao")
        infra_auditivas_visuais_p, infra_auditivas_visuais_a = extract_info(soup, "metadata-slug-infraestrutura-para-pessoas-com-deficiencias-auditivas-e-ou-visuais")
        outras_auditivas_visuais_p, outras_auditivas_visuais_a = extract_info(soup, "metadata-slug-outras-infraestrutura-para-pessoas-com-deficiencias-auditivas-e-ou-visuais")

        # Adicionar ao DataFrame
        result_df = pd.concat([result_df, pd.DataFrame([[
            museum_id,
            acessibilidade_fisica_p, acessibilidade_fisica_a,
            outras_locomocao_p, outras_locomocao_a,
            infra_auditivas_visuais_p, infra_auditivas_visuais_a,
            outras_auditivas_visuais_p, outras_auditivas_visuais_a
        ]], columns=columns)])
    
    except requests.RequestException as e:
        print(f"Erro ao acessar URL {url}: {e}")
        result_df = pd.concat([result_df, pd.DataFrame([[
            museum_id, "--", "--", "--", "--", "--", "--", "--", "--"
        ]], columns=columns)])
    except Exception as e:
        print(f"Erro ao processar URL {url}: {e}")

# Salvar o resultado em CSV
result_df.to_csv('museus_resultados.csv', index=False, encoding='utf-8')


# Tratando os dados obtidos

Juntando o df original com os dados do museus.gov.br com o df da raspagem em um único arquivo

In [ ]:
# Realizar a junção pela coluna "ID"
merged_df = pd.merge(result_df, df, on="ID", how="inner")

# Exibir os primeiros registros do DataFrame resultante
print(merged_df.head())

# Salvar o DataFrame unido em um arquivo CSV, se necessário
merged_df.to_csv('dados_museus.csv', index=False, encoding='utf-8')

In [25]:
df_museu.head(10)

,ID,Acessibilidade_Fisica_P,Acessibilidade_Fisica_A,Outras_Infraestruturas_Locomocao_P,Outras_Infraestruturas_Locomocao_A,Infraestrutura_Def_Auditivas_Visuais_P,Infraestrutura_Def_Auditivas_Visuais_A,Outras_Infraestruturas_Auditivas_Visuais_P,Outras_Infraestruturas_Auditivas_Visuais_A,Título,Esfera,Status de Funcionamento,Tipo,Temática,Estado,Município,Site Oficial,Link no MuseusBR
0,80119,Corrimão nas escadas e rampas|Rampa de acesso,Corrimão nas escadas e rampas|Rampa de acesso,- NÃO INFORMADO -,--,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,Casa da Cultura Dide Brandão,PÚBLICA – Municipal,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",SC,Itajaí,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_...
1,80120,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,--,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,--,Museu Sacro João XXIII,PRIVADA – Não informada,Aberto,Clássico/Tradicional,História,RN,Macaíba,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
2,80121,Elevador,Elevador,- NÃO INFORMADO -,--,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,Museu Paranaense de Ciências Forenses,PÚBLICA – Estadual/Distrital,Aberto,Clássico/Tradicional,Defesa e segurança pública,PR,Curitiba,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_...
3,117440,- NÃO INFORMADO -,- NÃO INFORMADO -,--,--,--,--,--,--,Museu da Cultura Hip Hop RS,PRIVADA – Associação,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",RS,Porto Alegre,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
4,80122,Corrimão nas escadas e rampas,Corrimão nas escadas e rampas,- NÃO INFORMADO -,--,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,"Museu Histórico, Antropológico, Arqueológico e...",PÚBLICA – Municipal,Aberto,Clássico/Tradicional,História,RS,Torres,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
5,80123,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,--,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,--,Museu Gramado Minerais e Pedras Preciosas,PRIVADA – Não informada,Aberto,- NÃO INFORMADO -,- NÃO INFORMADO -,RS,Canela,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
6,80124,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,--,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,--,Museu Centro de Valorização da Cultura Ibiapin...,PRIVADA – Não informada,Fechado,Clássico/Tradicional,"Educação, esporte e lazer",CE,Ibiapina,http://cevaciong.blogspot.com.br/,https://cadastro.museus.gov.br/?post_type=tnc_...
7,80125,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,Parque Estadual Matas do Segredo,PÚBLICA – Estadual/Distrital,Aberto,Unidade de conservação da natureza,"Ciências exatas, da terra, biológicas e da saúde",MS,Campo Grande (Mato Grosso do Sul),NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
8,80126,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,Memorial do Porto - Estação das Docas,PRIVADA – Não informada,Aberto,Clássico/Tradicional,História,PA,Belém (Pará),NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
9,80127,Corrimão nas escadas e rampas|Rampa de acesso|...,Corrimão nas escadas e rampas|Rampa de acesso|...,Vaga de estacionamento exclusiva para idosos,--,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,--,Museu do IMIP (Instituto de Medicina Integral ...,PRIVADA – Fundação,Aberto,Clássico/Tradicional,História,PE,Recife,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...


Verificando se existem valores diferentes nas colunas ("Acessibilidade_Fisica_P" e "Acessibilidade_Fisica_A") e ("Infraestrutura_Def_Auditivas_Visuais_P" e "Infraestrutura_Def_Auditivas_Visuais_A")

In [23]:
# Verificar se há diferenças entre as colunas
diferencas = df_museu[df_museu["Acessibilidade_Fisica_P"] != df_museu["Acessibilidade_Fisica_A"]]

# Exibir as linhas com diferenças, se existirem
if not diferencas.empty:
    print("Existem diferenças nas seguintes linhas:")
    print(diferencas)
else:
    print("Não há diferenças entre as colunas Acessibilidade_Fisica_P e Acessibilidade_Fisica_A.")

Não há diferenças entre as colunas Acessibilidade_Fisica_P e Acessibilidade_Fisica_A.


In [13]:
# Verificar se há diferenças entre as colunas
diferencas = df_museu[df_museu["Infraestrutura_Def_Auditivas_Visuais_P"] != df_museu["Infraestrutura_Def_Auditivas_Visuais_A"]]

# Exibir as linhas com diferenças, se existirem
if not diferencas.empty:
    print("Existem diferenças nas seguintes linhas:")
    print(diferencas)
else:
    print("Não há diferenças entre as colunas Infraestrutura_Def_Auditivas_Visuais_P e Infraestrutura_Def_Auditivas_Visuais_A.")

Não há diferenças entre as colunas Infraestrutura_Def_Auditivas_Visuais_P e Infraestrutura_Def_Auditivas_Visuais_A.


Verificando valores únicos em colunas

In [21]:
# Obter os valores únicos da coluna
valores_unicos = df_museu["Outras_Infraestruturas_Auditivas_Visuais_A"].unique()

# Exibir os valores únicos
print("Valores únicos na coluna 'Outras_Infraestruturas_Auditivas_Visuais_A':")
for valor in valores_unicos:
    print(valor)

Valores únicos na coluna 'Outras_Infraestruturas_Auditivas_Visuais_A':
--


Colunas "Outras_Infraestruturas_Locomocao_A" e "Outras_Infraestruturas_Auditivas_Visuais_A" = não possui valores diferentes de "--".

Portanto, podemos excluir as colunas:
- Outras_Infraestruturas_Locomocao_A = Não existem valores
- Outras_Infraestruturas_Auditivas_Visuais_A = Não existem valores
- Acessibilidade_Fisica_A = É igual a Acessibilidade_Fisica_P
- Infraestrutura_Def_Auditivas_Visuais_A = É igual a Infraestrutura_Def_Auditivas_Visuais_P


In [26]:

# Criar uma cópia do DataFrame
df_museu_tratado = df_museu.copy()

# Excluir as colunas especificadas
colunas_para_excluir = [
    "Outras_Infraestruturas_Locomocao_A",
    "Outras_Infraestruturas_Auditivas_Visuais_A",
    "Acessibilidade_Fisica_A",
    "Infraestrutura_Def_Auditivas_Visuais_A"
]

df_museu_tratado = df_museu_tratado.drop(columns=colunas_para_excluir)


In [27]:
df_museu_tratado.head()

,ID,Acessibilidade_Fisica_P,Outras_Infraestruturas_Locomocao_P,Infraestrutura_Def_Auditivas_Visuais_P,Outras_Infraestruturas_Auditivas_Visuais_P,Título,Esfera,Status de Funcionamento,Tipo,Temática,Estado,Município,Site Oficial,Link no MuseusBR
0,80119,Corrimão nas escadas e rampas|Rampa de acesso,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,Casa da Cultura Dide Brandão,PÚBLICA – Municipal,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",SC,Itajaí,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_...
1,80120,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,Museu Sacro João XXIII,PRIVADA – Não informada,Aberto,Clássico/Tradicional,História,RN,Macaíba,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
2,80121,Elevador,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,Museu Paranaense de Ciências Forenses,PÚBLICA – Estadual/Distrital,Aberto,Clássico/Tradicional,Defesa e segurança pública,PR,Curitiba,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_...
3,117440,- NÃO INFORMADO -,--,--,--,Museu da Cultura Hip Hop RS,PRIVADA – Associação,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",RS,Porto Alegre,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
4,80122,Corrimão nas escadas e rampas,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,"Museu Histórico, Antropológico, Arqueológico e...",PÚBLICA – Municipal,Aberto,Clássico/Tradicional,História,RS,Torres,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...


Renomeando colunas:
- "Acessibilidade_Fisica_P" para "Acessibilidade_fisica"
- "Outras_Infraestruturas_Locomocao_P" para "Infraestruturas_dificuldade_locomocao"
- "Infraestrutura_Def_Auditivas_Visuais_P" para "Infraestrutura_auditivas_visuais"
- "Outras_Infraestruturas_Auditivas_Visuais_P" para "Outras_infra_auditivas_visuais"

In [28]:
# Renomear as colunas no DataFrame
df_museu_tratado = df_museu_tratado.rename(columns={
    "Acessibilidade_Fisica_P": "Acessibilidade_fisica",
    "Outras_Infraestruturas_Locomocao_P": "Infraestruturas_dificuldade_locomocao",
    "Infraestrutura_Def_Auditivas_Visuais_P": "Infraestrutura_auditivas_visuais",
    "Outras_Infraestruturas_Auditivas_Visuais_P": "Outras_infra_auditivas_visuais"
})

In [29]:
df_museu_tratado.head()

,ID,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Título,Esfera,Status de Funcionamento,Tipo,Temática,Estado,Município,Site Oficial,Link no MuseusBR
0,80119,Corrimão nas escadas e rampas|Rampa de acesso,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,Casa da Cultura Dide Brandão,PÚBLICA – Municipal,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",SC,Itajaí,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_...
1,80120,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,Museu Sacro João XXIII,PRIVADA – Não informada,Aberto,Clássico/Tradicional,História,RN,Macaíba,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
2,80121,Elevador,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,Museu Paranaense de Ciências Forenses,PÚBLICA – Estadual/Distrital,Aberto,Clássico/Tradicional,Defesa e segurança pública,PR,Curitiba,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_...
3,117440,- NÃO INFORMADO -,--,--,--,Museu da Cultura Hip Hop RS,PRIVADA – Associação,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",RS,Porto Alegre,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
4,80122,Corrimão nas escadas e rampas,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,"Museu Histórico, Antropológico, Arqueológico e...",PÚBLICA – Municipal,Aberto,Clássico/Tradicional,História,RS,Torres,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...


In [30]:
# Salvar o DataFrame em um arquivo CSV
df_museu_tratado.to_csv('df_museu_tratado.csv', index=False, encoding='utf-8')

print("Arquivo 'df_museu_tratado.csv' salvo com sucesso!")


Arquivo 'df_museu_tratado.csv' salvo com sucesso!


In [4]:
# Configurar pandas para mostrar o conteúdo completo das células
pd.set_option('display.max_colwidth', None)  # Não limitar a largura das colunas
# pd.set_option('display.max_rows', None)      # Mostrar todas as linhas
# pd.set_option('display.max_columns', None)   # Mostrar todas as colunas

In [5]:
df = pd.read_csv('df_museu_tratado.csv', sep=',', encoding='UTF-8')
df.head()

,ID,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Título,Esfera,Status de Funcionamento,Tipo,Temática,Estado,Município,Site Oficial,Link no MuseusBR
0,80119,Corrimão nas escadas e rampas|Rampa de acesso,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,Casa da Cultura Dide Brandão,PÚBLICA – Municipal,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",SC,Itajaí,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119
1,80120,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,Museu Sacro João XXIII,PRIVADA – Não informada,Aberto,Clássico/Tradicional,História,RN,Macaíba,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80120
2,80121,Elevador,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,Museu Paranaense de Ciências Forenses,PÚBLICA – Estadual/Distrital,Aberto,Clássico/Tradicional,Defesa e segurança pública,PR,Curitiba,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80121
3,117440,- NÃO INFORMADO -,--,--,--,Museu da Cultura Hip Hop RS,PRIVADA – Associação,Aberto,Clássico/Tradicional,"Artes, arquitetura e linguística",RS,Porto Alegre,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=117440
4,80122,Corrimão nas escadas e rampas,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,"Museu Histórico, Antropológico, Arqueológico e Oceanográfico de Torres",PÚBLICA – Municipal,Aberto,Clássico/Tradicional,História,RS,Torres,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80122


In [6]:
# Reorganizar as colunas no novo DataFrame
colunas_ordenadas = [
    "ID",
    "Título",
    "Status de Funcionamento",
    "Estado",
    "Município",
    "Tipo",
    "Esfera",
    "Tipo Esfera",
    "Temática",
    "Acessibilidade_fisica",
    "Infraestruturas_dificuldade_locomocao",
    "Infraestrutura_auditivas_visuais",
    "Outras_infra_auditivas_visuais",
    "Site Oficial",
    "Link no MuseusBR"
]

# Criar o DataFrame reorganizado
df_organizado = df[colunas_ordenadas]

In [7]:
# Separar a coluna "Esfera" em duas novas colunas
df_organizado[["Esfera", "Tipo Esfera"]] = df_organizado["Esfera"].str.split("–", expand=True)

# Remover espaços extras ao redor dos valores nas novas colunas
df_organizado["Esfera"] = df_organizado["Esfera"].str.strip()
df_organizado["Tipo Esfera"] = df_organizado["Tipo Esfera"].str.strip()


In [8]:
df_organizado.head()

,ID,Título,Status de Funcionamento,Estado,Município,Tipo,Esfera,Temática,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Site Oficial,Link no MuseusBR,Tipo Esfera
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,"Artes, arquitetura e linguística",Corrimão nas escadas e rampas|Rampa de acesso,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119,Municipal
1,80120,Museu Sacro João XXIII,Aberto,RN,Macaíba,Clássico/Tradicional,PRIVADA,História,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80120,Não informada
2,80121,Museu Paranaense de Ciências Forenses,Aberto,PR,Curitiba,Clássico/Tradicional,PÚBLICA,Defesa e segurança pública,Elevador,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80121,Estadual/Distrital
3,117440,Museu da Cultura Hip Hop RS,Aberto,RS,Porto Alegre,Clássico/Tradicional,PRIVADA,"Artes, arquitetura e linguística",- NÃO INFORMADO -,--,--,--,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=117440,Associação
4,80122,"Museu Histórico, Antropológico, Arqueológico e Oceanográfico de Torres",Aberto,RS,Torres,Clássico/Tradicional,PÚBLICA,História,Corrimão nas escadas e rampas,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80122,Municipal


In [9]:
# Reorganizar as colunas no novo DataFrame
colunas_ordenadas = [
    "ID",
    "Título",
    "Status de Funcionamento",
    "Estado",
    "Município",
    "Tipo",
    "Esfera",
    "Tipo Esfera",
    "Temática",
    "Acessibilidade_fisica",
    "Infraestruturas_dificuldade_locomocao",
    "Infraestrutura_auditivas_visuais",
    "Outras_infra_auditivas_visuais",
    "Site Oficial",
    "Link no MuseusBR"
]

# Criar o DataFrame reorganizado
df_organizado = df_organizado[colunas_ordenadas]

In [11]:
df_organizado.head()

,ID,Título,Status de Funcionamento,Estado,Município,Tipo,Esfera,Tipo Esfera,Temática,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Site Oficial,Link no MuseusBR
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,Municipal,"Artes, arquitetura e linguística",Corrimão nas escadas e rampas|Rampa de acesso,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119
1,80120,Museu Sacro João XXIII,Aberto,RN,Macaíba,Clássico/Tradicional,PRIVADA,Não informada,História,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80120
2,80121,Museu Paranaense de Ciências Forenses,Aberto,PR,Curitiba,Clássico/Tradicional,PÚBLICA,Estadual/Distrital,Defesa e segurança pública,Elevador,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80121
3,117440,Museu da Cultura Hip Hop RS,Aberto,RS,Porto Alegre,Clássico/Tradicional,PRIVADA,Associação,"Artes, arquitetura e linguística",- NÃO INFORMADO -,--,--,--,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=117440
4,80122,"Museu Histórico, Antropológico, Arqueológico e Oceanográfico de Torres",Aberto,RS,Torres,Clássico/Tradicional,PÚBLICA,Municipal,História,Corrimão nas escadas e rampas,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80122


Expandindo as informações de acessibilidade

In [12]:
# Colunas que serão explodidas
colunas_para_explodir = [
    "Acessibilidade_fisica",
    "Infraestruturas_dificuldade_locomocao",
    "Infraestrutura_auditivas_visuais",
    "Outras_infra_auditivas_visuais"
]

# Criar uma cópia apenas com as colunas necessárias
df_explodido = df_organizado[["ID","Título","Status de Funcionamento","Estado","Município","Tipo","Esfera","Tipo Esfera","Temática"] + colunas_para_explodir + ["Site Oficial","Link no MuseusBR"]].copy()

# Explodir cada coluna que contém valores separados por "|"
for coluna in colunas_para_explodir:
    df_explodido[coluna] = df_explodido[coluna].str.split('|')  # Separar os valores
    df_explodido = df_explodido.explode(coluna)                # Expandir os valores para novas linhas

# Remover espaços extras nas colunas
for coluna in colunas_para_explodir:
    df_explodido[coluna] = df_explodido[coluna].str.strip()

# Visualizar o DataFrame resultante
print(df_explodido.head(30))


        ID  \
0    80119   
0    80119   
1    80120   
2    80121   
3   117440   
4    80122   
5    80123   
6    80124   
7    80125   
8    80126   
9    80127   
9    80127   
9    80127   
9    80127   
10   80128   
11   80129   
11   80129   
12   80131   
13   80132   
14   80130   
15   80134   
16   80135   
16   80135   
17   80133   
17   80133   
17   80133   
17   80133   
17   80133   
17   80133   
17   80133   

                                                                        Título  \
0                                                 Casa da Cultura Dide Brandão   
0                                                 Casa da Cultura Dide Brandão   
1                                                       Museu Sacro João XXIII   
2                                        Museu Paranaense de Ciências Forenses   
3                                                  Museu da Cultura Hip Hop RS   
4       Museu Histórico, Antropológico, Arqueológico e Oceanográfico de T

In [13]:
df_explodido.head(30)

,ID,Título,Status de Funcionamento,Estado,Município,Tipo,Esfera,Tipo Esfera,Temática,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Site Oficial,Link no MuseusBR
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,Municipal,"Artes, arquitetura e linguística",Corrimão nas escadas e rampas,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,Municipal,"Artes, arquitetura e linguística",Rampa de acesso,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119
1,80120,Museu Sacro João XXIII,Aberto,RN,Macaíba,Clássico/Tradicional,PRIVADA,Não informada,História,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80120
2,80121,Museu Paranaense de Ciências Forenses,Aberto,PR,Curitiba,Clássico/Tradicional,PÚBLICA,Estadual/Distrital,Defesa e segurança pública,Elevador,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80121
3,117440,Museu da Cultura Hip Hop RS,Aberto,RS,Porto Alegre,Clássico/Tradicional,PRIVADA,Associação,"Artes, arquitetura e linguística",- NÃO INFORMADO -,--,--,--,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=117440
4,80122,"Museu Histórico, Antropológico, Arqueológico e Oceanográfico de Torres",Aberto,RS,Torres,Clássico/Tradicional,PÚBLICA,Municipal,História,Corrimão nas escadas e rampas,- NÃO INFORMADO -,- NÃO POSSUI -,- NÃO POSSUI -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80122
5,80123,Museu Gramado Minerais e Pedras Preciosas,Aberto,RS,Canela,- NÃO INFORMADO -,PRIVADA,Não informada,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80123
6,80124,Museu Centro de Valorização da Cultura Ibiapinense,Fechado,CE,Ibiapina,Clássico/Tradicional,PRIVADA,Não informada,"Educação, esporte e lazer",- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,- NÃO INFORMADO -,http://cevaciong.blogspot.com.br/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80124
7,80125,Parque Estadual Matas do Segredo,Aberto,MS,Campo Grande (Mato Grosso do Sul),Unidade de conservação da natureza,PÚBLICA,Estadual/Distrital,"Ciências exatas, da terra, biológicas e da saúde",- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80125
8,80126,Memorial do Porto - Estação das Docas,Aberto,PA,Belém (Pará),Clássico/Tradicional,PRIVADA,Não informada,História,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,- NÃO POSSUI -,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80126


In [14]:
df_explodido.loc[df_explodido["ID"] == 80133]

,ID,Título,Status de Funcionamento,Estado,Município,Tipo,Esfera,Tipo Esfera,Temática,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Site Oficial,Link no MuseusBR
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Banheiros adaptados,- NÃO POSSUI -,Guia multimídia (audioguia com monitor),- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Banheiros adaptados,- NÃO POSSUI -,Maquetes táteis ou mapas em relevo,- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Banheiros adaptados,- NÃO POSSUI -,Obras e reproduções táteis,- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Banheiros adaptados,- NÃO POSSUI -,Sinalização tátil,- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Banheiros adaptados,- NÃO POSSUI -,Texto/Etiquetas em Braille,- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Banheiros adaptados,- NÃO POSSUI -,Tradutor de Língua Brasileira de Sinais (LIBRAS),- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Cadeira de rodas para uso do visitante,- NÃO POSSUI -,Guia multimídia (audioguia com monitor),- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Cadeira de rodas para uso do visitante,- NÃO POSSUI -,Maquetes táteis ou mapas em relevo,- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Cadeira de rodas para uso do visitante,- NÃO POSSUI -,Obras e reproduções táteis,- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133
17,80133,Museu Flamengo,Aberto,RJ,Rio de Janeiro,Clássico/Tradicional,PRIVADA,Associação,"Educação, esporte e lazer",Cadeira de rodas para uso do visitante,- NÃO POSSUI -,Sinalização tátil,- NÃO POSSUI -,https://museuflamengo.com/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80133


In [17]:
# Obter os valores únicos ordenados pelo número de ocorrências
valores_frequencia = df_explodido["Infraestrutura_auditivas_visuais"].value_counts()

# Exibir os valores únicos do mais citado ao menos citado
print(valores_frequencia)


Infraestrutura_auditivas_visuais
- NÃO POSSUI -                                      4264
- NÃO INFORMADO -                                   1772
Sinalização tátil                                   1221
Tradutor de Língua Brasileira de Sinais (LIBRAS)     798
Obras e reproduções táteis                           777
Texto/Etiquetas em Braille                           642
Maquetes táteis ou mapas em relevo                   613
Guia multimídia (audioguia com monitor)              564
--                                                    42
Name: count, dtype: int64


In [18]:
# Lista das colunas onde as substituições devem ser feitas
colunas_para_substituir = [
    "Acessibilidade_fisica",
    "Infraestruturas_dificuldade_locomocao",
    "Infraestrutura_auditivas_visuais",
    "Outras_infra_auditivas_visuais"
]

# Dicionário de substituições
substituicoes = {
    "- NÃO POSSUI -": "Não possui",
    "- NÃO INFORMADO -": "Não informado",
    "--": "Sem informação"
}

# Aplicar as substituições
for coluna in colunas_para_substituir:
    df_explodido[coluna] = df_explodido[coluna].replace(substituicoes)

# Exibir algumas linhas do DataFrame para verificar as mudanças
df_explodido.head()


,ID,Título,Status de Funcionamento,Estado,Município,Tipo,Esfera,Tipo Esfera,Temática,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Site Oficial,Link no MuseusBR
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,Municipal,"Artes, arquitetura e linguística",Corrimão nas escadas e rampas,Não informado,Não possui,Não possui,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,Municipal,"Artes, arquitetura e linguística",Rampa de acesso,Não informado,Não possui,Não possui,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119
1,80120,Museu Sacro João XXIII,Aberto,RN,Macaíba,Clássico/Tradicional,PRIVADA,Não informada,História,Não informado,Não informado,Não informado,Não informado,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80120
2,80121,Museu Paranaense de Ciências Forenses,Aberto,PR,Curitiba,Clássico/Tradicional,PÚBLICA,Estadual/Distrital,Defesa e segurança pública,Elevador,Não informado,Não possui,Não possui,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80121
3,117440,Museu da Cultura Hip Hop RS,Aberto,RS,Porto Alegre,Clássico/Tradicional,PRIVADA,Associação,"Artes, arquitetura e linguística",Não informado,Sem informação,Sem informação,Sem informação,NaN,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=117440


In [19]:
df_explodido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10693 entries, 0 to 4004
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ID                                     10693 non-null  int64 
 1   Título                                 10693 non-null  object
 2   Status de Funcionamento                10693 non-null  object
 3   Estado                                 10692 non-null  object
 4   Município                              10687 non-null  object
 5   Tipo                                   10686 non-null  object
 6   Esfera                                 10689 non-null  object
 7   Tipo Esfera                            10139 non-null  object
 8   Temática                               10688 non-null  object
 9   Acessibilidade_fisica                  10693 non-null  object
 10  Infraestruturas_dificuldade_locomocao  10693 non-null  object
 11  Infraestrutura_auditi

In [20]:
# Salvar o DataFrame em um arquivo CSV
df_explodido.to_csv("df_museu_tratado2.csv", index=False, encoding="utf-8")


In [21]:
df_agrupado = df_explodido.groupby('ID').agg({
    'Título': 'first',
    'Status de Funcionamento': 'first',
    'Estado': 'first',
    'Município': 'first',
    'Esfera': 'first',
    'Tipo': 'first',
    'Temática': 'first',
    'Acessibilidade_fisica': '|'.join,
    'Infraestruturas_dificuldade_locomocao': '|'.join,
    'Infraestrutura_auditivas_visuais': '|'.join,
    'Outras_infra_auditivas_visuais': '|'.join,
    'Link no MuseusBR': 'first',
    'Site Oficial': 'first'
}).reset_index()

df_agrupado.to_csv("df_museu_tratado3.csv", index=False, encoding="utf-8")

In [22]:
df_agrupado.head()

,ID,Título,Status de Funcionamento,Estado,Município,Esfera,Tipo,Temática,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Link no MuseusBR,Site Oficial
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,PÚBLICA,Clássico/Tradicional,"Artes, arquitetura e linguística",Corrimão nas escadas e rampas|Rampa de acesso,Não informado|Não informado,Não possui|Não possui,Não possui|Não possui,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80119,http://www.casadacultura.worpress.com
1,80120,Museu Sacro João XXIII,Aberto,RN,Macaíba,PRIVADA,Clássico/Tradicional,História,Não informado,Não informado,Não informado,Não informado,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80120,None
2,80121,Museu Paranaense de Ciências Forenses,Aberto,PR,Curitiba,PÚBLICA,Clássico/Tradicional,Defesa e segurança pública,Elevador,Não informado,Não possui,Não possui,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80121,https://www.policiacientifica.pr.gov.br/
3,80122,"Museu Histórico, Antropológico, Arqueológico e Oceanográfico de Torres",Aberto,RS,Torres,PÚBLICA,Clássico/Tradicional,História,Corrimão nas escadas e rampas,Não informado,Não possui,Não possui,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80122,None
4,80123,Museu Gramado Minerais e Pedras Preciosas,Aberto,RS,Canela,PRIVADA,- NÃO INFORMADO -,- NÃO INFORMADO -,Não informado,Não informado,Não informado,Não informado,https://cadastro.museus.gov.br/?post_type=tnc_col_208_item&p=80123,None


Nesse momento decidi não detalhar todos os recursos e infraestrutura de acessibilidade. Vou organizar isso depois, para o trabalho final do curso, onde posso me organizar e solicitar via LAI todas as informações e um dicionário de dados para ser mais preciso na análise.

Por enquanto seguirei apenas com os gráficos:
- Quantidade de museus abertos e fechados, agrupados por Público e privado.
- Quantidade de museus abertos e fechados, por região do brasil
- Quantidade de museus que possuem acessibilidade física, por região
- Quantidade de museus que possuem acessibilidade auditiva e visual, por região

In [2]:
df = pd.read_csv('df_museu_tratado2.csv', sep=',', encoding='UTF-8')
df.head()

,ID,Título,Status de Funcionamento,Estado,Município,Tipo,Esfera,Tipo Esfera,Temática,Acessibilidade_fisica,Infraestruturas_dificuldade_locomocao,Infraestrutura_auditivas_visuais,Outras_infra_auditivas_visuais,Site Oficial,Link no MuseusBR
0,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,Municipal,"Artes, arquitetura e linguística",Corrimão nas escadas e rampas,Não informado,Não possui,Não possui,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_...
1,80119,Casa da Cultura Dide Brandão,Aberto,SC,Itajaí,Clássico/Tradicional,PÚBLICA,Municipal,"Artes, arquitetura e linguística",Rampa de acesso,Não informado,Não possui,Não possui,http://www.casadacultura.worpress.com,https://cadastro.museus.gov.br/?post_type=tnc_...
2,80120,Museu Sacro João XXIII,Aberto,RN,Macaíba,Clássico/Tradicional,PRIVADA,Não informada,História,Não informado,Não informado,Não informado,Não informado,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...
3,80121,Museu Paranaense de Ciências Forenses,Aberto,PR,Curitiba,Clássico/Tradicional,PÚBLICA,Estadual/Distrital,Defesa e segurança pública,Elevador,Não informado,Não possui,Não possui,https://www.policiacientifica.pr.gov.br/,https://cadastro.museus.gov.br/?post_type=tnc_...
4,117440,Museu da Cultura Hip Hop RS,Aberto,RS,Porto Alegre,Clássico/Tradicional,PRIVADA,Associação,"Artes, arquitetura e linguística",Não informado,Sem informação,Sem informação,Sem informação,NaN,https://cadastro.museus.gov.br/?post_type=tnc_...


In [3]:
df_ori = df

Criei abaixo algumas bases mas acabei não incluindo nesse tabalho para não deixar a página longa demais. Decidi focar diretamente nos números de acessibilidade.

In [6]:
# 1. Número de museus por "Status de Funcionamento"
df_status = df.groupby("Status de Funcionamento")["ID"].nunique().reset_index(name="Total")
df_status.to_csv("StatusFuncionamento.csv", index=False)

In [7]:
# 2. Número de museus por "Status de Funcionamento" agrupados por "Estado"
df_func_estado = df.groupby(["Estado", "Status de Funcionamento"])["ID"].nunique().reset_index(name="Total")
df_func_estado.to_csv("dfs/FuncionamentoPorEstado.csv", index=False)

In [8]:
# 3. Número de museus por "Status de Funcionamento" agrupados por região
# Criando um dicionário de estados para regiões do Brasil
estados_regioes = {
    "AC": "Norte",
    "AL": "Nordeste",
    "AP": "Norte",
    "AM": "Norte",
    "BA": "Nordeste",
    "CE": "Nordeste",
    "DF": "Centro-Oeste",
    "ES": "Sudeste",
    "GO": "Centro-Oeste",
    "MA": "Nordeste",
    "MT": "Centro-Oeste",
    "MS": "Centro-Oeste",
    "MG": "Sudeste",
    "PA": "Norte",
    "PB": "Nordeste",
    "PR": "Sul",
    "PE": "Nordeste",
    "PI": "Nordeste",
    "RJ": "Sudeste",
    "RN": "Nordeste",
    "RS": "Sul",
    "RO": "Norte",
    "RR": "Norte",
    "SC": "Sul",
    "SP": "Sudeste",
    "SE": "Nordeste",
    "TO": "Norte"
}
df["Região"] = df["Estado"].map(estados_regioes)
df_func_regiao = df.groupby(["Região", "Status de Funcionamento"])["ID"].nunique().reset_index(name="Total")
df_func_regiao.to_csv("dfs/FuncionamentoPorRegiaoBrasil.csv", index=False)

In [9]:
# 4. Número de museus por "Status de Funcionamento", "Esfera" e "Tipo Esfera"
df_func_esfera = df.groupby(["Esfera", "Tipo Esfera", "Status de Funcionamento"])["ID"].nunique().reset_index(name="Total")
df_func_esfera.to_csv("dfs/FuncionamentoPorEsfera.csv", index=False)

In [14]:
# 5. Número de museus por "Acessibilidade_fisica", agrupados por "Status de Funcionamento" e "Estado"
df_acessibilidade = df.groupby(
    ["Acessibilidade_fisica", "Estado", "Status de Funcionamento"]
)["ID"].nunique().reset_index(name="Total")
df_acessibilidade.to_csv("dfs/So_Acessibilidade_fisica.csv", index=False)

# 5. Número de museus por "Infraestruturas_dificuldade_locomocao", agrupados por "Status de Funcionamento" e "Estado"
df_acessibilidade = df.groupby(
    ["Infraestruturas_dificuldade_locomocao", "Estado", "Status de Funcionamento"]
)["ID"].nunique().reset_index(name="Total")
df_acessibilidade.to_csv("dfs/So_Infra_fisica.csv", index=False)

In [11]:
# 6. Número de museus por "Infraestrutura_auditivas_visuais" e "Outras_infra_auditivas_visuais", agrupados por "Status de Funcionamento" e "Estado"
df_infraestrutura = df.groupby(
    ["Infraestrutura_auditivas_visuais", "Outras_infra_auditivas_visuais", "Status de Funcionamento", "Estado"]
)["ID"].nunique().reset_index(name="Total")
df_infraestrutura.to_csv("dfs/Infraestrutura_auditivas_visuais.csv", index=False)

Separando infos sobre acessibilidade

In [33]:
df_info_ac = pd.read_csv('df_museu_tratado.csv', sep=',', encoding='UTF-8')

In [24]:
# Filtrar museus que têm "- NÃO POSSUI -" em todas as 4 colunas
nao_possui_filter = (df_info_ac["Acessibilidade_fisica"] == "- NÃO POSSUI -") & \
                    (df_info_ac["Infraestruturas_dificuldade_locomocao"] == "- NÃO POSSUI -") & \
                    (df_info_ac["Infraestrutura_auditivas_visuais"] == "- NÃO POSSUI -") & \
                    (df_info_ac["Outras_infra_auditivas_visuais"] == "- NÃO POSSUI -")

df_nao_possui = df_info_ac[nao_possui_filter]
df_nao_possui.to_csv("dfs/1-Museus_Nao_Possui.csv", index=False)
df_nao_possui.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1265 entries, 7 to 3857
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ID                                     1265 non-null   int64 
 1   Acessibilidade_fisica                  1265 non-null   object
 2   Infraestruturas_dificuldade_locomocao  1265 non-null   object
 3   Infraestrutura_auditivas_visuais       1265 non-null   object
 4   Outras_infra_auditivas_visuais         1265 non-null   object
 5   Título                                 1265 non-null   object
 6   Esfera                                 1265 non-null   object
 7   Status de Funcionamento                1265 non-null   object
 8   Tipo                                   1265 non-null   object
 9   Temática                               1265 non-null   object
 10  Estado                                 1265 non-null   object
 11  Município             

In [26]:
# Filtrar museus que têm "- NÃO INFORMADO -" ou "--" em todas as 4 colunas
nao_info_filter = ((df_info_ac["Acessibilidade_fisica"] == "- NÃO INFORMADO -") | (df_info_ac["Acessibilidade_fisica"] == "--")) & \
                  ((df_info_ac["Infraestruturas_dificuldade_locomocao"] == "- NÃO INFORMADO -") | (df_info_ac["Infraestruturas_dificuldade_locomocao"] == "--")) & \
                  ((df_info_ac["Infraestrutura_auditivas_visuais"] == "- NÃO INFORMADO -") | (df_info_ac["Infraestrutura_auditivas_visuais"] == "--")) & \
                  ((df_info_ac["Outras_infra_auditivas_visuais"] == "- NÃO INFORMADO -") | (df_info_ac["Outras_infra_auditivas_visuais"] == "--"))

df_nao_info = df_info_ac[nao_info_filter]
df_nao_info.to_csv("dfs/1-Museus_Nao_Informado.csv", index=False)
df_nao_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1183 entries, 1 to 3993
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ID                                     1183 non-null   int64 
 1   Acessibilidade_fisica                  1183 non-null   object
 2   Infraestruturas_dificuldade_locomocao  1183 non-null   object
 3   Infraestrutura_auditivas_visuais       1183 non-null   object
 4   Outras_infra_auditivas_visuais         1183 non-null   object
 5   Título                                 1183 non-null   object
 6   Esfera                                 1183 non-null   object
 7   Status de Funcionamento                1183 non-null   object
 8   Tipo                                   1183 non-null   object
 9   Temática                               1183 non-null   object
 10  Estado                                 1183 non-null   object
 11  Município             

In [34]:
# Filtrar museus que têm pelo menos 1 valor diferente de "--", "- NÃO INFORMADO -" ou "- NÃO POSSUI -"
nao_possui_info_filter = ((~df_info_ac["Acessibilidade_fisica"].isin(["- NÃO INFORMADO -", "--", "- NÃO POSSUI -"])) | 
                          (~df_info_ac["Infraestruturas_dificuldade_locomocao"].isin(["- NÃO INFORMADO -", "--", "- NÃO POSSUI -"])) | 
                          (~df_info_ac["Infraestrutura_auditivas_visuais"].isin(["- NÃO INFORMADO -", "--", "- NÃO POSSUI -"])) | 
                          (~df_info_ac["Outras_infra_auditivas_visuais"].isin(["- NÃO INFORMADO -", "--", "- NÃO POSSUI -"])))

df_possui_info = df_info_ac[nao_possui_info_filter]
df_possui_info.to_csv("dfs/1-Museus_Com_Informacao_Valida.csv", index=False)
df_possui_info.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1511 entries, 0 to 4004
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ID                                     1511 non-null   int64 
 1   Acessibilidade_fisica                  1511 non-null   object
 2   Infraestruturas_dificuldade_locomocao  1511 non-null   object
 3   Infraestrutura_auditivas_visuais       1511 non-null   object
 4   Outras_infra_auditivas_visuais         1511 non-null   object
 5   Título                                 1511 non-null   object
 6   Esfera                                 1510 non-null   object
 7   Status de Funcionamento                1511 non-null   object
 8   Tipo                                   1508 non-null   object
 9   Temática                               1510 non-null   object
 10  Estado                                 1511 non-null   object
 11  Município             

verificar IDs que não estão contabilizados

In [39]:
# Passo 1: Extrair os valores da coluna "ID" de cada dataframe e criar o dataframe "df_IDs"
df_nao_possui_ids = df_nao_possui["ID"]
df_nao_info_ids = df_nao_info["ID"]
df_possui_info_ids = df_possui_info["ID"]

# Concatenar os IDs em um único dataframe e garantir que seja um DataFrame com a coluna "ID"
df_IDs = pd.concat([df_nao_possui_ids, df_nao_info_ids, df_possui_info_ids]).drop_duplicates().reset_index(drop=True)
df_IDs = pd.DataFrame(df_IDs, columns=["ID"])  # Garantir que seja um DataFrame com a coluna "ID"

# Passo 2: Verificar quais IDs de df_info_ac não estão em df_IDs
missing_ids = df_info_ac[~df_info_ac["ID"].isin(df_IDs["ID"])]

# Passo 3: Incluir as colunas de interesse no DataFrame "missing_ids"
missing_ids = missing_ids[["ID", "Acessibilidade_fisica","Infraestruturas_dificuldade_locomocao","Infraestrutura_auditivas_visuais","Outras_infra_auditivas_visuais","Título","Esfera","Status de Funcionamento","Tipo","Temática","Estado","Município","Site Oficial","Link no MuseusBR"]]

# Passo 4: Salvar o resultado em um CSV
missing_ids.to_csv("dfs/1-Museus_Nao_possui_ou_nao_informou.csv", index=False)

# Mostrar o resultado (opcional)
missing_ids.info()


<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, 15 to 3999
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ID                                     46 non-null     int64 
 1   Acessibilidade_fisica                  46 non-null     object
 2   Infraestruturas_dificuldade_locomocao  46 non-null     object
 3   Infraestrutura_auditivas_visuais       46 non-null     object
 4   Outras_infra_auditivas_visuais         46 non-null     object
 5   Título                                 46 non-null     object
 6   Esfera                                 45 non-null     object
 7   Status de Funcionamento                46 non-null     object
 8   Tipo                                   43 non-null     object
 9   Temática                               45 non-null     object
 10  Estado                                 45 non-null     object
 11  Município              

Continuar criando os gráficos sobre acessibilidade

In [27]:
# Carregando o DataFrame
df_testt = pd.read_csv('df_museu_tratado.csv', sep=',', encoding='UTF-8')

# Filtrando as linhas onde a coluna "Estado" está vazia ou sem registro
linhas_sem_estado = df_testt[df_testt['Estado'].isna() | (df_testt['Estado'] == '')]

# Exibindo o resultado
print(linhas_sem_estado)

##### A linha com ID 127321 de PORTOALEGRE estava sem o estado!!!!
##### Por isso o total de museus não batia com o painel do site, estava contabilizando 4004.

          ID Acessibilidade_fisica Infraestruturas_dificuldade_locomocao  \
3870  127321        - NÃO POSSUI -                                    --   

     Infraestrutura_auditivas_visuais Outras_infra_auditivas_visuais  \
3870                   - NÃO POSSUI -                             --   

                       Título                Esfera Status de Funcionamento  \
3870  Museu Numismático NUMIS  PRIVADA – Associação                  Aberto   

                      Tipo  Temática Estado     Município  \
3870  Clássico/Tradicional  História    NaN  Porto Alegre   

                   Site Oficial  \
3870  https://www.numis.mus.br/   

                                       Link no MuseusBR  
3870  https://cadastro.museus.gov.br/?post_type=tnc_...  


In [29]:
df_c_info = pd.read_csv('dfs/1-Museus_Com_Informacao_Valida.csv', sep=',', encoding='UTF-8')
df_n_informado = pd.read_csv('dfs/1-Museus_Nao_Informado.csv', sep=',', encoding='UTF-8')
df_n_possui = pd.read_csv('dfs/1-Museus_Nao_Possui - Copia.csv', sep=',', encoding='UTF-8')
df_n_info_ou_possui = pd.read_csv('dfs/1-Museus_Nao_possui_ou_nao_informou.csv', sep=',', encoding='UTF-8')

Adicionando regiões

In [30]:
# Dicionário para mapear estados para suas regiões
estado_para_regiao = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Adicionando a coluna de região
df_c_info['Região'] = df_c_info['Estado'].map(estado_para_regiao)
df_n_informado['Região'] = df_n_informado['Estado'].map(estado_para_regiao)
df_n_possui['Região'] = df_n_possui['Estado'].map(estado_para_regiao)
df_n_info_ou_possui['Região'] = df_n_info_ou_possui['Estado'].map(estado_para_regiao)

# Contando o número de museus por região: com acessibilidade
df_museus_por_regiao1 = df_c_info.groupby('Região').size().reset_index(name='Quantidade de Museus')
df_museus_por_regiao1.to_csv("dfs/2-Museus_com_acessibilidade_por_regiao.csv", index=False)

# Contando o número de museus por região: não informado
df_museus_por_regiao2 = df_n_informado.groupby('Região').size().reset_index(name='Quantidade de Museus')
df_museus_por_regiao2.to_csv("dfs/2-Museus_n_informado_por_regiao.csv", index=False)

# Contando o número de museus por região: não possui
df_museus_por_regiao3 = df_n_possui.groupby('Região').size().reset_index(name='Quantidade de Museus')
df_museus_por_regiao3.to_csv("dfs/2-Museus_n_possui_por_regiao.csv", index=False)

# Contando o número de museus por região: não informou ou não possui
df_museus_por_regiao4 = df_n_info_ou_possui.groupby('Região').size().reset_index(name='Quantidade de Museus')
df_museus_por_regiao4.to_csv("dfs/2-Museus_n_info_ou_possui_por_regiao.csv", index=False)

Adicionando regiões e consolidadndo

In [31]:
# Dicionário para mapear estados para suas regiões
estado_para_regiao = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Função para adicionar a coluna de região e calcular quantidade de museus por região
def processar_dataframe(df, categoria):
    df['Região'] = df['Estado'].map(estado_para_regiao)
    agrupado = df.groupby('Região').size().reset_index(name='Quantidade de Museus')
    agrupado['Categoria'] = categoria
    return agrupado

# Processando cada DataFrame
df1 = processar_dataframe(df_c_info, "Possui recursos ou infraestrutura")
df2 = processar_dataframe(df_n_informado, "Não há informação disponível para consultar")
df3 = processar_dataframe(df_n_possui, "Não possui nenhum recurso ou infraestrutura")
df4 = processar_dataframe(df_n_info_ou_possui, "Não é possível classificar")

# Concatenando os resultados
df_consolidado = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Salvando o resultado em um único CSV
df_consolidado.to_csv("dfs/2-Museus_por_regiao_consolidado.csv", index=False)


Por região

In [32]:
import pandas as pd

# Dicionário para mapear estados para suas regiões
estado_para_regiao = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Função para preparar os DataFrames com agrupamento
def processar_dataframe(df, categoria):
    df['Região'] = df['Estado'].map(estado_para_regiao)
    agrupado = (
        df.groupby(['Região', 'Status de Funcionamento'])
        .size()
        .reset_index(name=categoria)
    )
    return agrupado

# Processando cada DataFrame
df1 = processar_dataframe(df_c_info, "Com acessibilidade")
df2 = processar_dataframe(df_n_possui, "Não possui")
df3 = processar_dataframe(df_n_informado, "Não informado")
df4 = processar_dataframe(df_n_info_ou_possui, "Não informado ou não possui")

# Unindo os DataFrames em um único DataFrame
df_consolidado = pd.merge(df1, df2, on=['Região', 'Status de Funcionamento'], how='outer')
df_consolidado = pd.merge(df_consolidado, df3, on=['Região', 'Status de Funcionamento'], how='outer')
df_consolidado = pd.merge(df_consolidado, df4, on=['Região', 'Status de Funcionamento'], how='outer')

# Preenchendo valores ausentes com zero
df_consolidado.fillna(0, inplace=True)

# Salvando o resultado em um único CSV
df_consolidado.to_csv("dfs/2-Museus_por_regiao_status_consolidado.csv", index=False)


Por estado

In [33]:
import pandas as pd

# Função para preparar os DataFrames com agrupamento
def processar_dataframe(df, categoria):
    agrupado = (
        df.groupby(['Estado', 'Status de Funcionamento'])
        .size()
        .reset_index(name=categoria)
    )
    return agrupado

# Processando cada DataFrame
df1 = processar_dataframe(df_c_info, "Com acessibilidade")
df2 = processar_dataframe(df_n_possui, "Não possui")
df3 = processar_dataframe(df_n_informado, "Não informado")
df4 = processar_dataframe(df_n_info_ou_possui, "Não informado ou não possui")

# Unindo os DataFrames em um único DataFrame
df_consolidado = pd.merge(df1, df2, on=['Estado', 'Status de Funcionamento'], how='outer')
df_consolidado = pd.merge(df_consolidado, df3, on=['Estado', 'Status de Funcionamento'], how='outer')
df_consolidado = pd.merge(df_consolidado, df4, on=['Estado', 'Status de Funcionamento'], how='outer')

# Preenchendo valores ausentes com zero
df_consolidado.fillna(0, inplace=True)

# Salvando o resultado em um único CSV
df_consolidado.to_csv("dfs/3-Museus_por_estado_status_consolidado.csv", index=False)


por esfera

In [34]:


# Função para preparar os DataFrames com agrupamento
def processar_dataframe(df, categoria):
    agrupado = (
        df.groupby(['Esfera', 'Status de Funcionamento'])
        .size()
        .reset_index(name=categoria)
    )
    return agrupado

# Processando cada DataFrame
df1 = processar_dataframe(df_c_info, "Com acessibilidade")
df2 = processar_dataframe(df_n_possui, "Não possui")
df3 = processar_dataframe(df_n_informado, "Não informado")
df4 = processar_dataframe(df_n_info_ou_possui, "Não informado ou não possui")

# Unindo os DataFrames em um único DataFrame
df_consolidado = pd.merge(df1, df2, on=['Esfera', 'Status de Funcionamento'], how='outer')
df_consolidado = pd.merge(df_consolidado, df3, on=['Esfera', 'Status de Funcionamento'], how='outer')
df_consolidado = pd.merge(df_consolidado, df4, on=['Esfera', 'Status de Funcionamento'], how='outer')

# Preenchendo valores ausentes com zero
df_consolidado.fillna(0, inplace=True)

# Salvando o resultado em um único CSV
df_consolidado.to_csv("dfs/4-Museus_por_esfera_status_consolidado.csv", index=False)
